In [182]:
import pandas as pd

data = pd.read_csv("TokensTest.csv", index_col = 0)
keyTokens = pd.read_csv("NeighbourhoodCountsTrain.csv")
antiTokens = pd.read_csv("AntiNeighbourCountTrain.csv", header = None)
#antiTokens = antiTokens.iloc[:,0].values

print (data.head())
print (keyTokens.head(10))
print (antiTokens.head())

for i in range(len(keyTokens["N=1"])):
    if (keyTokens["N=1"][0:i].sum()/float(keyTokens["N=1"].sum()) >=0.7):
        print ("---")
        print(i)
        break

                                                    0
0   bed eight michael wu forty eight year under dr...
1   vera abbott bed four under dr liu come in with...
2   cindy benedictos bed under dr smtih come in fo...
4   ramanathan nisha pillai bed year old under dr ...
11  rebecca birks bed seven yr old under dalianis ...
   Unnamed: 0  N=1  N=2  N=3  N=4  N=5  N=6  N=7   N=8   N=9
0          mg  262  407  492  614  763  851  953  1068  1155
1          on  152  268  315  383  447  505  553   595   625
2       daily  113  187  256  318  379  419  482   527   570
3         day   58  101  124  170  217  245  282   323   352
4  medication   46   61   84  112  139  161  185   204   222
5         for   46   91  127  158  189  220  263   303   341
6          qd   42   62   82   98  120  129  146   162   173
7         prn   40   56   73   92  111  124  136   152   159
8     regular   40   51   60   65   72   73   83    88    91
9         bid   39   68   92  118  145  166  191   210   225
     

In [178]:
# getting to see if the words are nouns, adverbs, adjectives
from nltk.corpus import wordnet as wn

# WordNet POS tags are: NOUN = 'n', ADJ = 's', VERB = 'v', ADV = 'r', ADJ_SAT = 'a'
# Descriptions (c) https://web.stanford.edu/~jurafsky/slp3/10.pdf
tag_map = {
        'JJ':wn.ADJ, # adjective (yellow)                         
        'NN':wn.NOUN, # noun, sing. or mass (llama)          
        'NNS':wn.NOUN, # noun, plural (llamas)                  
}

def get_wordnet_pos(tag):
    try:
        if tag_map[tag] != None:
            return tag_map[tag]
    except:
        return None
    return None

In [179]:
import nltk

def GetNoun (tokens):
    tokens = nltk.pos_tag(tokens)
    output = set()
    for i in tokens:
        if get_wordnet_pos(i[1]) == wn.NOUN:
            output.add(i[0])
    if len(output) >= 0:
        return list(output)

In [187]:
def GetMeds (neighbourhood = 1, numberOfWords = 1):
    meds = [[]]
    counter = 0
    for i in range(len(data)):
        tokenized = data.iloc[i,0].split(" ")
        nouns = []
        for keyToken in keyTokens.iloc[:73,0]:
            try:
                while keyToken in tokenized:
                    keyLocation = tokenized.index(keyToken)
                    tokenWindow = tokenized[max(keyLocation-neighbourhood,0):keyLocation+neighbourhood+1]
                    #tokenWindow = tokenized[keyLocation:keyLocation+neighbourhood+1]
                    tokenWindow.remove(keyToken)
                    for token in tokenWindow:
                        if token in antiTokens:
                            counter += 1
                            break
                    nouns += GetNoun(tokenWindow)
                    tokenized.pop(keyLocation)
            except ValueError:
                pass
        meds += [[",".join(list(set(nouns)))]]  
    print ("tokens removed:",counter)
    return meds
            

In [188]:
import time
startTime = time.time()
meds = GetMeds(3)
endTime = time.time()
print (endTime-startTime)

tokens removed: 0
13.400266647338867


In [193]:
medsDF = pd.DataFrame(meds)
medsDF = medsDF.iloc[1:]
medsDF.index = data.index
medsDF.to_csv("tokenminer_results.csv", encoding = "utf-8")

In [38]:
data["TextToken"][1]

'vera abbott bed four under dr liu come in with chest pain with history of stroke previous chest pain asthma cataract glaucoma almost blind need assistance nitros with effect still under monitoring'

In [77]:
tags = nltk.pos_tag(['vera', 'abbott', 'bed', 'four', 'under'])
print (tags)
print (GetNoun(['vera', 'abbott', 'bed', 'four', 'under']))

[('vera', 'NN'), ('abbott', 'NN'), ('bed', 'VBD'), ('four', 'CD'), ('under', 'IN')]
{'vera'}
{'vera', 'abbott'}


TypeError: '>=' not supported between instances of 'set' and 'int'

In [150]:
if "of" in antiTokens.iloc[:,0].values:
    print ("yes")

yes
